In [1]:
'''
In the following we will explore a number of techniques, including padding and strided convolutions, 
that offer more control over the size of the output. As motivation, note that since kernels generally have width and height greater than 1, 
after applying many successive convolutions, we tend to wind up with outputs that are considerably smaller than our input. If we start with a 240*240 pixel image, 
ten layers of 5*5 convolutions reduce the image to 200*200 pixels, slicing off 30% of the image and with it obliterating any interesting information on the boundaries of the original image. 
Padding is the most popular tool for handling this issue. In other cases, we may want to reduce the dimensionality drastically, e.g.,
if we find the original input resolution to be unwieldy. Strided convolutions are a popular technique that can help in these instances.
'''
import torch
from torch import nn

In [ ]:
'''
卷积神经网络通常使用具有奇数高度和宽度值的卷积核，例如 1，3，5 或 7。选择奇数尺寸的卷积核有如下优势：

1. **对称性**：奇数尺寸卷积核允许在空间上保持对称性。每个维度的卷积核大小使我们可以在上下方向上添加相同数量的行，并在左右方向上添加相同数量的列。因此，卷积核的中心像素正好位于相邻像素的中心，这有助于保持输入和输出特征图之间的空间关系。

2. **保持尺寸不变**：使用奇数尺寸的卷积核，我们可以通过添加与卷积核大小相适应的填充，轻松地保持输入和输出特征图的空间维度不变。例如，在应用 3x3 的卷积核时，我们可以在输入图像的边界周围添加一个像素的填充，以确保输出特征图与输入具有相同的尺寸。

3. **局部连接**：奇数尺寸的卷积核有助于捕获输入图像中局部区域的信息。由于以中心像素为基准建立卷积核，这简化了捕捉局部信息和结构的过程。

综上所述，在卷积神经网络中使用奇数尺寸的卷积核有助于保持输入和输出特征图之间的空间关系，并允许网络更好地捕获和保留局部图像信息。 
'''